In [2]:
import bcrypt
from pymongo import MongoClient
import datetime
from getpass import getpass

In [3]:
client = MongoClient("mongodb://localhost:27017")
db = client["edgecloud_project"]

print("Connected database: ", db.name)

Connected database:  edgecloud_project


In [4]:
users = db["users"]

In [5]:
users.create_index("username", unique=True)

'username_1'

In [6]:
def create_user():
    # check if username is available
    while True:
        username = input("\nEnter username: ").strip().lower()

        if not username:
            print("Username cannot be empty.")
            continue

        if users.find_one({"username": username}):
            print("❌ Username already in use, try another.")
        else:
            print("✅ Username available.")
            break

    # password selection and hashing
    password = getpass("Enter a password: ")

    if len(password) < 4:
        print("⚠️ Password must be at least 4 characters.")
        return

    hashed_pw = bcrypt.hashpw(password.encode("utf-8"), bcrypt.gensalt())

    # ask for user description
    description = input("Describe yourself in few lines: ").strip()

    # updating user in db
    new_user = {
        "username": username,
        "password_hash": hashed_pw.decode("utf-8"),
        "raw_description": description,
        "traits": None,
        "created_at": datetime.datetime.utcnow(),
        "last_login": None
    }

    insert_result = users.insert_one(new_user)

    print(f"\n🎉 User created successfully with ID: {insert_result.inserted_id}\n")
    return insert_result.inserted_id

In [7]:
def login():

    username = input("\nEnter username: ").strip().lower()
    password = getpass("Enter password: ")

    # Look up user
    user = users.find_one({"username": username})

    if not user:
        print("No such user. Try again.")
        return None

    # Check password match
    stored_hash = user["password_hash"].encode("utf-8")

    if bcrypt.checkpw(password.encode("utf-8"), stored_hash):
        print("Login successful!\nLoading profile...")

        # Update last login timestamp
        users.update_one(
            {"username": username},
            {"$set": {"last_login": datetime.datetime.utcnow()}}
        )

        # Return useful info (not password hash)
        return {
            "user_id": str(user["_id"]),
            "username": user["username"],
            "traits": user["traits"],
            "raw_description": user["raw_description"]
        }

    else:
        print("Incorrect password. Try again.")
        return None

In [8]:
def change_password(username: str):
    user = users.find_one({"username": username.lower()})
    if not user:
        print("User not found.")
        return

    old_pw = getpass("Enter current password: ")
    if not bcrypt.checkpw(old_pw.encode("utf-8"), user["password_hash"].encode("utf-8")):
        print("Incorrect current password.")
        return

    new_pw = getpass("Enter new password: ")
    if len(new_pw) < 4:
        print("New password too short.")
        return

    new_hash = bcrypt.hashpw(new_pw.encode("utf-8"), bcrypt.gensalt()).decode("utf-8")
    users.update_one({"_id": user["_id"]}, {"$set": {"password_hash": new_hash}})
    print("Password updated.")

In [9]:
def login_loop():
    while True:
        result  = login()
        if result:
            return result

In [10]:
create_user()


Enter username:  shreenath


✅ Username available.


Enter a password:  ········
Describe yourself in few lines:  I am Computer Science student, dreaming of becoming a ML Engineer or AI Engineer, currently in dire need of a job and searching for it like anything.



🎉 User created successfully with ID: 692e8707036679b7ef220f21



/var/folders/_b/rvmhd42s13q_mt4jvjz0nq8h0000gn/T/ipykernel_69513/1111500623.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.datetime.utcnow(),


ObjectId('692e8707036679b7ef220f21')

In [11]:
login()


Enter username:  shreenath
Enter password:  ········


Login successful!
Loading profile...


/var/folders/_b/rvmhd42s13q_mt4jvjz0nq8h0000gn/T/ipykernel_69513/1432218085.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"$set": {"last_login": datetime.datetime.utcnow()}}


{'user_id': '692e8707036679b7ef220f21',
 'username': 'shreenath',
 'traits': None,
 'raw_description': 'I am Computer Science student, dreaming of becoming a ML Engineer or AI Engineer, currently in dire need of a job and searching for it like anything.'}

In [13]:
login()


Enter username:  shreenath
Enter password:  ········


Incorrect password. Try again.


In [17]:
change_password("shreenath")

Enter current password:  ········
Enter new password:  ········


Password updated.
